# Sesión 7.2 Transformers sentence embeddings

En esta sesión se muestra un ejemplo de funcionamiento de los sentence embeddings basados en Transformers BERT y RoBERTa.

1.  Instalamos librería sentence-transformers y otras librerías necesarias.
2.  Definimos una serie de textos con los que vamos a trabajar.
3.  Descargamos un modelo sentence-transformers y a partir de él creamos otro modelo que usaremos para medir similitud entre los textos previamente definidos.
4.  Descargamos datasets de tweets (train y test) y los codificamos (encode) con el modelo creado en el punto anterior, generando así los embeddings para entrenamiento y prueba.
5.  Con estos conjuntos de embeddings train y test entrenamos y probamos un clasificador SVM. Es decir, hemos usado un modelo para crear los sentence embeddings, pero usamos un modelo diferente para clasificarlos. Los embeddings que genera el modelo sentence-transformer actúan como características para entrenar a otro modelo más sencillo.
6. Usamos el clasificador para hacer inferencias.


## BERT sentence embeddigns
Se puede calcular el vector que representa un texto completo usando los sentence embeddings. Para hacerlo, instalaremos la libraría "sentence-transformers" de SBERT.net

Existe una serie de modelos que se han pre-entrenado para mejorar tareas sobre sentencias, que incluyen un amplio rango de aplicaciones como la 'búsqueda semántica', 'la similitud semántica entre textos' y el 'paraphrase mining'.

La librería sentence-transformers facilita la manpulación de estos modelos.

Aunque también se pueden utilizar otros modelos de BERT y RoBERTa preentrenados más generales para trabajar con oraciones, los modelos "sentence transformers" se han entrenado específicamente para ellas.

Se puede encontrar información y modelos pre-entrenados en https://www.sbert.net/docs/pretrained_models.html

In [ ]:
# Instalamos primero la librería necesaria
!pip3 install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
# Importamos otras librerías ...
from sentence_transformers import SentenceTransformer, models
from torch import nn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Definimos un conjunto de textos sobre los que vamos a calcular sus embeddings
textos=['El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. \
Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje.\
Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'El procesamiento del lenguaje natural (NLP, por sus siglas en inglés) es una rama de la inteligencia artificial que ayuda a las computadoras a entender, interpretar y manipular el lenguaje humano. \
NLP toma elementos prestados de muchas disciplinas, incluyendo la ciencia de la computación y la lingüística computacional, en su afán por cerrar la brecha entre la comunicación humana y el entendimiento de las computadoras."""], """El procesamiento del lenguaje natural (PLN o NLP) es un campo dentro de la inteligencia artificial y la lingüística aplicada que estudia las interacciones mediante uso del lenguaje natural entre los seres humanos y las máquinas. Más concretamente se centra en el procesamiento de las comunicaciones humanas, dividiéndolas en partes, e identificando los elementos más relevantes del mensaje. Con la Comprensión y Generación de Lenguaje Natural, busca que las máquinas consigan entender, interpretar y manipular el lenguaje humano.'
, 'La lingüística computacional es un campo interdisciplinario que se ocupa del desarrollo de formalismos del funcionamiento del lenguaje natural, tales que puedan ser transformados en programas ejecutables para un ordenador. \
Dicho desarrollo se sitúa entre el modelado basado en reglas y el modelado estadístico del lenguaje natural desde una perspectiva computacional, y en él participan lingüistas e informáticos especializados en inteligencia artificial, psicólogos cognoscitivos y expertos en lógica, entre otros.'
, 'El aprendizaje automático es un tipo de inteligencia artificial (AI) que proporciona a las computadoras la capacidad de aprender, sin ser programadas explícitamente. El aprendizaje automático se centra en el desarrollo de programas informáticos que pueden cambiar cuando se exponen a nuevos datos.'
, 'El  aprendizaje profundo es un tema que cada vez adquiere mayor relevancia en el campo de la inteligencia artificial (IA). Siendo una subcategoría del aprendizaje automático, el aprendizaje profundo trata del uso de redes neuronales para mejorar cosas tales como el reconocimiento de voz, la visión por ordenador y el procesamiento del lenguaje natural. \
Rápidamente se está convirtiendo en uno de los campos más solicitados en informática. \
En los últimos años, el aprendizaje profundo ha ayudado a lograr avances en áreas tan diversas como la percepción de objetos, el procesamiento del lenguaje natural y el reconocimiento de voz (todas ellas áreas especialmente complejas para los investigadores en IA).',
'El coste de la energía va a subir mucho los próximos meses y la población va a tener que pagar cantidades excesivas a las eléctricas']


In [ ]:
# CREAMOS UN MODELO PARA MEDIR SIMILITUD DE TEXTOS

# 1.- Cogemos un modelo sentence transformers entrenado para medir similitud
#     entre textos en español.
sentence_transformers_path_model = "hiiamsid/sentence_similarity_spanish_es"

# 2.- Creamos un modelo de sentence transformers a partir de una lista de
#     módulos que se llaman secuencialemente para crear el modelo.
#     Creamos los módulos ...

# 2.1 .- Modelo (módulo) para generar los embeddings
#        (truncamos secuencia de entrada a 768 tokens)
word_embedding_model = models.Transformer(sentence_transformers_path_model,
                                          max_seq_length=768)

# 2.2.- Modelo para generar un sentence embedding de longitud fija a partir
#       de sentencias de longitud variable.
pooling_model = models.Pooling(word_embedding_model.
                                    get_word_embedding_dimension())

# 2.3.- Modelo para pasar el sentence embedding a través de una FFN:
#       This layer takes a fixed-sized sentence embedding and passes it through
#       a feed-forward layer.
dense_model = models.Dense(
    # size of the input dimension
    in_features=pooling_model.get_sentence_embedding_dimension(),

    # Output size
    out_features=768,
    activation_function=nn.Tanh())

# 2.4.- Finalmente creamos el modelo SentenceTransformer
model = SentenceTransformer(modules=[word_embedding_model,
                                     pooling_model,
                                     dense_model])

# OBSERVACIONES:
# ------------------------------------------------------------------------------
# 1.- podríamos haber cargado directamente el modelo sin necesidad de
# crearlo a partir de los módulos.
#
# model = SentenceTransformer(sentence_transformers_path_model)
#
# Esta es la mejor opción cuando se cuenta, como es el caso, con modelos ya
# ajustados a trabajar con oraciones y no hay necesidad de personalizar la
# arquitectura.
#
# ------------------------------------------------------------------------------
# 2.- también podríamos haber usado un modelo base de BERT o RoBERTa
# sentence_transformers_path_model = 'CenIA/distillbert-base-spanish-uncased'
#

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Probamos los BERT Sentence embeddins para
calcular la similitud ente distintos textos.

In [ ]:
# OBTENEMOS LOS EMBEDDINGS DE LOS TEXTOS usando el modelo que acabamos de
# obtener
# Una vez que tenemos el modelo lo usamos ...

# 1.- Obtenenemos los embeddings de los textos
embeddings = model.encode(textos)
print(embeddings.shape)
print(embeddings)

# 2.- Calculamos la similitud de los textos (de sus sentence embeddings)
results = cosine_similarity(embeddings[1::],
                            embeddings[0].reshape(1,-1)).reshape(-1,) # Op -- (n_docs,1) -- Cosine Sim with each doc
print('\n',results)

(6, 768)
[[-0.42546558 -0.04559131  0.44809163 ...  0.39955157  0.10775524
  -0.17105728]
 [-0.34658775 -0.0064696   0.37754294 ...  0.47506446  0.07919478
  -0.1855971 ]
 [-0.20230211 -0.06166632 -0.12575275 ...  0.10118747 -0.08078835
   0.15100352]
 [-0.5108904  -0.01209015  0.4074086  ...  0.21271248  0.1116633
   0.1622303 ]
 [-0.30661666  0.11043147  0.24577947 ...  0.2547182   0.22697638
  -0.22979765]
 [-0.29811427 -0.6086222   0.36659443 ... -0.3475118  -0.25459397
   0.34171534]]

 [0.9598114  0.74622536 0.69575584 0.7961874  0.06111154]


Entrenamiento de un clasificador de tweets.
Probamos a entrenar el clasificador de tuits de prácticas anteriores. Para ello:

1.- Obtenemos las sentencias (tweets).

2.- Obtenemos sus sentence embeddings.

3.- Entrenamos un modelo SVC con los embeddings de entrenamiento.

4.- Evaluamos el modelo con el conjunto de test.

In [ ]:
# Repositorio UMU
# Descargamos el fichero de datasetEspañol.csv
!wget -c --no-check-certificate http://valencia.inf.um.es/valencia-plne/dataset_train.csv
!wget -c --no-check-certificate http://valencia.inf.um.es/valencia-plne/dataset_test.csv

data_dir_path = ""

--2025-03-22 10:40:26--  http://valencia.inf.um.es/valencia-plne/dataset_train.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:80... failed: Connection timed out.
Retrying.

--2025-03-22 10:42:36--  (try: 2)  http://valencia.inf.um.es/valencia-plne/dataset_train.csv
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:80... failed: Connection timed out.
Retrying.

--2025-03-22 10:44:48--  (try: 3)  http://valencia.inf.um.es/valencia-plne/dataset_train.csv
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:80... failed: Connection timed out.
Retrying.

--2025-03-22 10:47:00--  (try: 4)  http://valencia.inf.um.es/valencia-plne/dataset_train.csv
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:80... failed: Connection timed out.
Retrying.

--2025-03-22 10:49:16--  (try: 5)  http://valencia.inf.um.es/valencia-plne/dataset_train.csv
Connectin

In [ ]:
# En caso de que haya descargado los ficheros en una computadora local
# asociada a drive.

from google.colab import drive
g_drive_path = "/content/drive"
drive.mount(g_drive_path)

# Directorio Drive
data_dir_path = "/content/drive/Othercomputers/your_path"


Mounted at /content/drive
total 23
drwx------ 2 root root 4096 Mar 24 15:06 p1_5
drwx------ 2 root root 4096 Mar 25 07:32 p1_6
drwx------ 2 root root 4096 Mar 24 10:33 p1_7
drwx------ 2 root root 4096 Feb 26 08:13 P2
-rw------- 1 root root 2294 Mar  5 11:33 spain_reddit.json
drwx------ 2 root root 4096 Feb 26 08:13 T3


Generamos los sentences embeddings de los tuits de entrenamiento y validación.
Este proceso tarda mucho en CPU por lo que es recomendable usar un entorno con GPU.

In [ ]:
import json
import csv

import pandas
from sklearn.svm import LinearSVC

df_train = pandas.read_csv(data_dir_path + "dataset_train.csv",encoding="UTF-8")
df_test = pandas.read_csv(data_dir_path + "dataset_test.csv",encoding="UTF-8")

# Ponemos en lower_case los dos conjuntos de tweets
df_train.tweet = df_train.tweet.apply(lambda x: x.lower())
df_test.tweet = df_test.tweet.apply(lambda x: x.lower())

# Obtenemos los sentence embeddings de los conjuntos de entrenamiento y prueba.
sentence_train = model.encode(df_train.tweet.tolist())
sentence_test = model.encode((df_test.tweet.tolist()))

In [ ]:
print(df_train.head())
print("------------------------------------------------")
print(df_test.head())

            twitter_id   twitter_created_at  \
0  1248859034721148929  2020-04-11 10:22:51   
1  1248937166773923840  2020-04-11 15:33:19   
2  1239278483991339009  2020-03-15 20:53:09   
3  1245862884493553667  2020-04-03 01:57:13   
4  1247584775877144577  2020-04-07 19:59:24   

                                               tweet  \
0  #buenosdias ☁️? en este #sabadodegloria #malag...   
1  "paseando al perro". un vecino de torreaguera ...   
2              #quedateencasa. juntos lo conseguimos   
3  a lo ke llega ste #gobgenocida #socialcomunism...   
4  ⛹️‍♀️? aunque creas que tu vida en un deshecho...   

                                corpus             user  agreement  votes  \
0  Estado de alarma nacional (oficial)  Malagaconacento        100      1   
1  Estado de alarma nacional (oficial)    NACHOGARCIA09        100      1   
2  Estado de alarma nacional (oficial)        jnrafael1        100      1   
3  Estado de alarma nacional (oficial)          MARLE68        100      

Entrenamos un modelo SVM para clasificar las oraciones.

Es decir, hemos usado un modelo para crear los sentence embeddings, pero vamos a usar ahora un modelo diferente para clasificarlos.

Este es un enfoque bastante habitual y que evita el coste de ajustar el modelo neuronal para una tarea determinada. Los embeddings que genera el modelo actúan como características para entrenar otro modelo más sencillo (usamos los transformadores como "extractores de características").

In [ ]:
# Entrenamos el mismo SVM que en sesiones anteriores
clf_sentence_embeddings = LinearSVC(random_state=0, tol=1e-5).fit(sentence_train, df_train.label)
predicted = clf_sentence_embeddings.predict(sentence_test)

accuracy = np.mean(predicted == df_test.label)
print("Resultados Sentence Embeddings ----- Accuracy:", accuracy)
from sklearn import metrics
print(metrics.classification_report(df_test.label, predicted))

Resultados Sentence Embeddings ----- Accuracy: 0.8355704697986577
              precision    recall  f1-score   support

    negative       0.76      0.70      0.73       561
    positive       0.87      0.90      0.88      1227

    accuracy                           0.84      1788
   macro avg       0.81      0.80      0.80      1788
weighted avg       0.83      0.84      0.83      1788



Probamos algunos ejemplos de inferencia con el modelo ya entrenado

In [ ]:
# Probamos algunos ejemplos con los modelos inferidos
textos = ['hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]
# Codificamos estos documentos
textos_SE = model.encode(textos)
# Predecimos
predicted = clf_sentence_embeddings.predict(textos_SE)

# Imprimimos los textos y su predicción para TF
for doc, category_tf in zip(textos, predicted):
  print('TF: %r => %s' % (doc, category_tf))

TF: 'hay muchos más muertos por covid' => negative
TF: 'el número de afectados por covid aumenta' => negative
TF: 'vamos a salir de la pandemia' => positive
TF: 'ánimo a todos' => positive
